In [1]:
from sklearn import svm
import numpy as np
import os
import numpy
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
import collections
from sklearn.metrics import f1_score
import warnings
warnings.filterwarnings('ignore')
def SVM(UserName,Window=256,StartTime=0,EndTime=63):
    file_path = os.path.join("ChannelsExtracted",UserName, 'ArousalEncoded'+".csv")
    ArousalFile =  np.genfromtxt(fname=file_path,delimiter=',').astype(int)
    file_path = os.path.join("ChannelsExtracted",UserName, 'ValenceEncoded'+".csv")
    ValenceFile =  np.genfromtxt(fname=file_path,delimiter=',').astype(int)
    BandsFile = np.genfromtxt('training/'+str(StartTime)+str(EndTime)+'/'+str(Window)+UserName+'.csv',delimiter=',')
    X_train, X_test, y_train, y_test , z_train,z_test= train_test_split(BandsFile, ValenceFile,ArousalFile, test_size=0.2)

# Feature Scaling
    from sklearn.preprocessing import StandardScaler
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)  	

    
# SVM Classifier Valence
    clf = SVC(kernel = 'poly')
    clf.fit(X_train, y_train)
    y_predict = clf.predict(X_test)
    cm = confusion_matrix(y_test, y_predict)
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)
    	
# SVM Classifier Arousal
    clf = SVC(kernel = 'poly')
    clf.fit(X_train, z_train)
    z_predict = clf.predict(X_test)
    return [accuracy_score(z_test, z_predict)*100,accuracy_score(y_test, y_predict)*100
            ,f1_score(z_test, z_predict, average='weighted'),
            f1_score(y_test, y_predict, average='weighted')]


    


####################### Happy Sad ###################
def SVMHS(UserName,Window=256,StartTime=0,EndTime=63):
    file_path = os.path.join("ChannelsExtracted",UserName, 'HappyAndSadEncoded'+".csv")
    SurveyFile =  np.genfromtxt(fname=file_path,delimiter=',').astype(int)
    BandsFile = np.genfromtxt('training/'+str(StartTime)+str(EndTime)+'/'+str(Window)+UserName+'HS'+'.csv',delimiter=',')
    X_train, X_test, y_train, y_test = train_test_split(BandsFile, SurveyFile, test_size=0.2)
    while ((np.sum(y_train == 0) == 0) or (np.sum(y_train == 1) == 0)): #Don't Start training unless you have 2 classes.. happens with imbalance
        print("here")
        X_train, X_test, y_train, y_test = train_test_split(BandsFile, SurveyFile, test_size=0.2)
        
    from sklearn.preprocessing import StandardScaler
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)  	
    clf = SVC(kernel = 'rbf')
    clf.fit(X_train, y_train)
    y_predict = clf.predict(X_test)
    cm = confusion_matrix(y_test, y_predict)
    return [accuracy_score(y_test, y_predict)*100,
            f1_score(y_test, y_predict, average='weighted')]
###########All emotions #####################

def SVMCombined(UserName,Window=256,StartTime=0,EndTime=63):
    file_path = os.path.join("ChannelsExtracted",UserName, 'ValenceAndArousalEncoded'+".csv")
    SurveyFile =  np.genfromtxt(fname=file_path,delimiter=',').astype(int)
    BandsFile = np.genfromtxt('training/'+str(StartTime)+str(EndTime)+'/'+str(Window)+UserName+'.csv',delimiter=',')
    X_train, X_test, y_train, y_test = train_test_split(BandsFile, SurveyFile, test_size=0.2)
    from sklearn.preprocessing import StandardScaler
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)  	
    clf = SVC(kernel = 'rbf')
    clf.fit(X_train, y_train)
    y_predict = clf.predict(X_test)
    cm = confusion_matrix(y_test, y_predict)
    return [accuracy_score(y_test, y_predict)*100,
            f1_score(y_test, y_predict, average='weighted')]

################ Run Each Method Multiple Times #####################
def Run100(UserName,Window=256,StartTime=0,EndTime=63):
    arousal=[]
    valence=[]
    F1_ScoreArousal=[]
    F1_ScoreValence=[]
    for i in range (100):
        svm=SVM(UserName,Window,StartTime,EndTime)
        arousal+=[svm[0]]
        valence+=[svm[1]]
        F1_ScoreArousal+=[svm[2]]
        F1_ScoreValence+=[svm[3]]
    print("Arousal "+str(np.mean(arousal)))
    print(collections.Counter(arousal))
    print("Valence "+str(np.mean(valence)))
    print(collections.Counter(valence))
    return [np.mean(arousal),np.mean(valence),np.mean(F1_ScoreArousal),np.mean(F1_ScoreValence)];
    

    
def RunCombined100(UserName,Window=256,StartTime=0,EndTime=63):
    Combined=[]
    F1_ALL=[]
    for i in range (100):
        AllEmotions=SVMCombined(UserName,Window,StartTime,EndTime)
        Combined+=[AllEmotions[0]]
        F1_ALL+=[AllEmotions[1]]
    print("Array of average All Emotions accurecies for user "+UserName)
    print(collections.Counter(Combined))
    return [np.mean(Combined),np.mean(F1_ALL)];
    
def RunHS100(UserName,Window=256,StartTime=0,EndTime=63):
    Combined=[]
    F1_HS=[]
    for i in range (100):
        HS=SVMHS(UserName,Window,StartTime,EndTime)
        Combined+=[HS[0]]
        F1_HS+=[HS[1]]
    print("Array of average Happy Sad accurecies for user "+UserName)
    print(collections.Counter(Combined))
    return [np.mean(Combined),np.mean(F1_HS)];

##################Main#########################
if __name__ == "__main__":
    HSAccurecy=0;
    HSF1=0;
    CombinedAccurecy=0;
    CombinedF1=0;
    ValenceAccurecy=0;
    ValenceF1=0;
    ArousalAccurecy=0;
    ArousalF1=0;
    for i in range(1,10):
        print("s0"+str(i))
        #Happy and Sad
        var =RunHS100("s0"+str(i))
        print("Happy Sad")
        print(var)
        HSAccurecy+=var[0]
        HSF1+=var[1]
        #All 4 Emotions
        var1 =RunCombined100("s0"+str(i))
        print("Combined Emotions")
        print(var1)
        CombinedAccurecy+=var1[0]
        CombinedF1+=var1[1]
        #Seperated Valence and Arousal
        var2=Run100("s0"+str(i))
        print("Arousal Valence")
        print(var2)
        ArousalAccurecy+=var2[0]
        ValenceAccurecy+=var2[1]
        ArousalF1+=var2[2]
        ValenceF1+=var2[3]
    for i in range(10,33):
        print("s"+str(i))
        #Happy and Sad
        if not ("s"+str(i) == "s31"):
            var =RunHS100("s"+str(i))
            print("Happy Sad")
            print(var)
            HSAccurecy+=var[0]
            HSF1+=var[1]
        #All 4 Emotions
        var1 =RunCombined100("s"+str(i))
        print("Combined Emotions")
        print(var1)
        CombinedAccurecy+=var1[0]
        CombinedF1+=var1[1]
        #Seperated Valence and Arousal
        var2=Run100("s"+str(i))
        print("Arousal Valence")
        print(var2)
        ArousalAccurecy+=var2[0]
        ValenceAccurecy+=var2[1]
        ArousalF1+=var2[2]
    print("Accurecy happy sad for 31 users "+str(HSAccurecy/31))
    print("F1 happy sad for  31 users "+str(HSF1/31))
    print("Accurecy Combined sad for 32 users"+str(CombinedAccurecy/32))
    print("F1 Combined sad for  32 users "+str(CombinedF1/32))
    print("Accurecy Valence for  32 users "+str(ValenceAccurecy/32))
    print("F1 Valence for 32 users "+str(ValenceF1/32))
    print("Accurecy Arousal for  32 users "+str(ArousalAccurecy/32))
    print("F1 Arousal for  32 users "+str(ArousalF1/32))


s01
Array of average Happy Sad accurecies for user s01
Counter({40.0: 46, 60.0: 40, 80.0: 7, 20.0: 6, 0.0: 1})
Happy Sad
[49.2, 0.41322222222222216]
Array of average All Emotions accurecies for user s01
Counter({37.5: 32, 25.0: 31, 50.0: 16, 12.5: 13, 0.0: 5, 62.5: 3})
Combined Emotions
[31.25, 0.2342816558441559]
Arousal 59.0
Counter({62.5: 35, 50.0: 25, 75.0: 21, 37.5: 8, 25.0: 6, 87.5: 5})
Valence 36.125
Counter({37.5: 37, 25.0: 37, 50.0: 16, 62.5: 5, 12.5: 3, 75.0: 2})
Arousal Valence
[59.0, 36.125, 0.4812571595071596, 0.2587366522366523]
s02
here
Array of average Happy Sad accurecies for user s02
Counter({100.0: 44, 80.0: 42, 60.0: 9, 40.0: 5})
Happy Sad
[85.0, 0.7905952380952379]
Array of average All Emotions accurecies for user s02
Counter({50.0: 38, 37.5: 23, 62.5: 18, 25.0: 13, 75.0: 6, 12.5: 2})
Combined Emotions
[46.875, 0.34029039016539014]
Arousal 65.25
Counter({62.5: 32, 75.0: 30, 50.0: 21, 87.5: 11, 37.5: 3, 25.0: 2, 100.0: 1})
Valence 74.125
Counter({75.0: 34, 62.5: 28,

Array of average All Emotions accurecies for user s15
Counter({37.5: 34, 25.0: 26, 50.0: 20, 12.5: 15, 62.5: 2, 75.0: 2, 0.0: 1})
Combined Emotions
[33.875, 0.2817589285714286]
Arousal 48.0
Counter({50.0: 28, 62.5: 27, 37.5: 15, 25.0: 14, 12.5: 7, 75.0: 6, 87.5: 3})
Valence 57.75
Counter({62.5: 31, 50.0: 22, 75.0: 19, 37.5: 18, 87.5: 5, 25.0: 4, 100.0: 1})
Arousal Valence
[48.0, 57.75, 0.4145804473304473, 0.4808075258075257]
s16
Array of average Happy Sad accurecies for user s16
Counter({80.0: 33, 100.0: 24, 60.0: 23, 40.0: 14, 20.0: 5, 0.0: 1})
Happy Sad
[70.8, 0.6657460317460316]
Array of average All Emotions accurecies for user s16
Counter({25.0: 27, 37.5: 26, 12.5: 20, 50.0: 17, 62.5: 5, 0.0: 3, 87.5: 2})
Combined Emotions
[32.375, 0.2748819444444444]
Arousal 46.625
Counter({50.0: 32, 37.5: 31, 62.5: 17, 25.0: 10, 75.0: 5, 12.5: 3, 87.5: 2})
Valence 59.75
Counter({50.0: 28, 62.5: 27, 75.0: 17, 37.5: 16, 87.5: 9, 100.0: 2, 25.0: 1})
Arousal Valence
[46.625, 59.75, 0.3763863358863359

Array of average Happy Sad accurecies for user s29
Counter({83.33333333333334: 38, 66.66666666666666: 28, 100.0: 14, 50.0: 13, 33.33333333333333: 7})
Happy Sad
[73.16666666666666, 0.6684960317460318]
Array of average All Emotions accurecies for user s29
Counter({50.0: 35, 62.5: 31, 37.5: 15, 75.0: 13, 25.0: 5, 12.5: 1})
Combined Emotions
[53.625, 0.41461384448884453]
Arousal 68.875
Counter({75.0: 27, 62.5: 27, 87.5: 21, 50.0: 18, 100.0: 3, 37.5: 3, 25.0: 1})
Valence 60.0
Counter({50.0: 34, 62.5: 24, 75.0: 21, 37.5: 10, 87.5: 8, 25.0: 2, 100.0: 1})
Arousal Valence
[68.875, 60.0, 0.6102608502608502, 0.5258533966033965]
s30
Array of average Happy Sad accurecies for user s30
Counter({71.42857142857143: 33, 85.71428571428571: 27, 57.14285714285714: 24, 42.857142857142854: 10, 100.0: 3, 28.57142857142857: 3})
Happy Sad
[68.57142857142856, 0.5764152514152513]
Array of average All Emotions accurecies for user s30
Counter({50.0: 33, 62.5: 30, 75.0: 12, 37.5: 12, 25.0: 8, 87.5: 4, 12.5: 1})
Comb